In [ ]:
#| default_exp domain.user

In [ ]:
#| export
import time
from dataclasses import dataclass
import fastlite as fl
from meme_games.core import *

In [ ]:

# | export
@dataclass
class User(Model):
    uid: str
    name: str
    avatar: Optional[str] = None

    async def set_picture(self, file: UploadFile, content_path='./user-content'):
        """Sets the user's profile picture."""
        self.reset_picture(content_path)
        fname = f"{self.uid}_{time.time()}.{file.filename.split('.')[-1]}"
        dir = Path(content_path)
        dir.mkdir(parents=True, exist_ok=True)
        with open(dir/fname, 'wb') as f: f.write(await file.read())
        self.avatar = str(fname)

    def reset_picture(self, content_path='./user-content'):
        """Removes the user's profile picture."""
        if not self.avatar: return
        path = Path(content_path)/self.avatar
        path.unlink(missing_ok=True)
        self.avatar = None
        
    def __eq__(self, value: 'User'): return self.uid == value.uid

In [ ]:
#| export
class UserManager(DataManager):
    """Manages user data and database operations."""
    def _set_tables(self): 
        self.db.create_table('user', User.columns(), pk='uid', transform=True)
        self.db.t.user.cls = User
        self.users: fl.Table = self.db.t.user
        return self.users
    
    def create(self, uid: str=None, name: str = 'null'):
        u = User(uid or random_id(), name)
        self.users.insert(u)
        return u
    
    def get(self, uid: str) -> User: return self.users.get(uid)
    def update(self, user: User) -> User: return self.users.update(user)
    
    def get_or_create(self, sess: dict, name: str = 'null') -> User:
        """Gets a user from the database or creates a new one based on the session."""
        sess = sess['session'] if 'session' in sess else sess
        uid = sess.setdefault('uid', random_id())
        if uid in self.users: return self.users[uid]
        return self.create(uid, name)

In [ ]:
#|export
DI.register_service(UserManager)

In [ ]:
#| export
def user_beforeware(manager: UserManager, skip=None):
    '''Makes sure that request always contains valid user'''
    def before(req: Request): req.state.user = manager.get_or_create(req.session)
    return Beforeware(before, skip)

In [ ]:
db = init_db()
user_manager = UserManager(db)
bware = user_beforeware(user_manager)

In [ ]:
hdrs = [
    Script(src="https://unpkg.com/hyperscript.org@0.9.13"),
    Link(rel="stylesheet", href="https://fonts.googleapis.com/icon?family=Material+Icons")
]
app, rt = fast_app(pico=False, before=bware, hdrs=hdrs)

In [ ]:
server = JupyUvi(app)

In [ ]:
server.stop()